# OpenVINO Docker Usage

## Goal
This tutorial will guide you through step-by-step instructions using OpenVINO docker container
* Step 1: Download a Keras-Tensorflow model and Freeze the model
* Step 2: Download OpenVINO docker container
* Step 3: Convert the Tensorflow Frozen model into OpenVINO model using the OpenVINO model optimizer via Docker
* Step 4: Run Inference using OpenVINO Inference Engine via Docker
* Step 5: Benchmark with OpenVINO benchmark app via Docker

## Prerequisites
1. **Install** Tensorflow and Keras.
1. **Install** Docker CE
To install docker on  Linux:

```
curl -fsSL https://get.docker.com -o get-docker.sh &&
sudo sh get-docker.sh && 
sudo usermod -aG docker $USER
```


- Click [here](https://docs.docker.com/install/linux/docker-ce/ubuntu) for complete Ubuntu instructions. For other OS platforms, see [here](https://docs.docker.com/install/).
- Setup docker to be used as a non-root user, to run `docker` commands without `sudo` . 
```sudo usermod -aG docker `whoami` ```
- Exit and restart your SSH session so that your username is in effect in the docker group.
- After exiting and restarting your SSH session, you should be able to run `docker` commands without `sudo`.
- Test installation by running ```	docker run hello-world ```

**NOTE**: If your machine is behind a proxy, See **HTTP/HTTPS proxy** section [here](https://docs.docker.com/config/daemon/systemd/)



## Step 1: Download a Keras-Tensorflow model and Freeze the model

In [1]:
# import the necessary packages

import tensorflow as tf
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
import tensorflow.keras.backend as K
import keras
import numpy as np
import shutil, sys, os
from pathlib import Path

Using TensorFlow backend.


In [2]:
keras_model = keras.applications.inception_v3.InceptionV3(include_top=True, weights='imagenet', input_tensor=None, input_shape=None, pooling=None, classes=1000)

### Freeze the model

In [3]:
pwd = !pwd
model_output_path = pwd[0] + "/"
frozen_model_name = 'keras_tf_model_frozen.pb'
frozen_model_path = model_output_path + "/frozen_model"

if os.path.isdir(frozen_model_path):
    print (frozen_model_path, "exists already. Deleting the folder")
    shutil.rmtree(frozen_model_path)

tf.get_logger().setLevel('INFO')

#K.clear_session()
K._LEARNING_PHASE = tf.constant(0)
K.set_learning_phase(False)
K.set_learning_phase(0)
K.set_image_data_format('channels_last')

# if you have an exiting Keras model, then do the following to load.
# keras_model_path = model_output_path + model_fname
# frozen_model_name = str(Path(keras_model_path).name) + '.pb'
# keras_model = tf.keras.models.load_model(keras_model_path, compile=False)

num_output = len(keras_model.outputs)

predictions = [None] * num_output
pred_node_names = [None] * num_output

for i in range(num_output):
    pred_node_names[i] = 'output_node' + str(i)
    predictions[i] = tf.identity(keras_model.outputs[i], name=pred_node_names[i])

sess = K.get_session()
sess.run(tf.global_variables_initializer())


/home/ec2-user/SageMaker/ravi//frozen_model exists already. Deleting the folder


##### Make Sure the the output node name is the same, below you can see `output_node0` is the same

In [4]:
print(predictions)
print(pred_node_names)

[<tf.Tensor 'output_node0:0' shape=(?, 1000) dtype=float32>]
['output_node0']


In [5]:
with sess.as_default():
    print(' --- Extracting default graph...')
    def_graph = sess.graph.as_graph_def()
    print(' --- Converting variables to constants...')
    constant_graph = tf.compat.v1.graph_util.convert_variables_to_constants(sess, def_graph, pred_node_names)
    print(' --- Removing training nodes...')
    infer_graph = tf.compat.v1.graph_util.remove_training_nodes(constant_graph) 
    print(' --- Saving Frozen graph...')
    graph_io.write_graph(infer_graph, frozen_model_path, frozen_model_name, as_text=False)

#print("\nModel input = ", keras_model_path)
print("Model input name = ", keras_model.input.op.name)
print("Model input shape = ", keras_model.input.shape)
print("Number of model outputs = ", num_output)
print("Model output name = ", keras_model.output.op.name)
print("Model output shape = ", keras_model.output.shape)
print("TensorFlow Frozen model is saved in: {}/{}".format(frozen_model_path, frozen_model_name))

 --- Extracting default graph...
 --- Converting variables to constants...
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 378 variables.
INFO:tensorflow:Converted 378 variables to const ops.
 --- Removing training nodes...
Instructions for updating:
Use `tf.compat.v1.graph_util.remove_training_nodes`
 --- Saving Frozen graph...
Model input name =  input_1
Model input shape =  (?, 299, 299, 3)
Number of model outputs =  1
Model output name =  predictions/Softmax
Model output shape =  (?, 1000)
TensorFlow Frozen model is saved in: /home/ec2-user/SageMaker/ravi//frozen_model/keras_tf_model_frozen.pb


## Step 2: Download OpenVINO docker container

In [6]:
# Download OpenVINO container
!docker pull iotgvmc/openvino

Using default tag: latest
latest: Pulling from iotgvmc/openvino
Digest: sha256:273c3429bd14f12da6aabee0c4c2afb69fc7b031265fb61c0325328208bdd1a0
Status: Image is up to date for iotgvmc/openvino:latest


## Step 3: Convert the Tensorflow Frozen model into OpenVINO model using the OpenVINO model optimizer via Docker

In [7]:
# Convert TF Frozen graph into OpenVINO IR using OV model optimizer via Docker container.

frozen_model = frozen_model_path + "/" + frozen_model_name
ovData_type = "FP32"
ovModel_path = "IR_models/" + ovData_type + "/"
docker_mnt_path = "/mnt/"
docker_frozen_model_path = docker_mnt_path + "frozen_model/" + frozen_model_name
docker_OvModel_Path = docker_mnt_path + "/" + ovModel_path
ovModel_name = "ov_inceptionv3"

model_inp_shape = keras_model.input.get_shape().as_list()
#set batch size
model_inp_shape[0] = 1
model_inp_shape = str(model_inp_shape).replace(" ", "")

if not os.path.exists(frozen_model):
    print(frozen_model + ' doesn\'t exist. Please make sure you have a trained keras to TF frozen model')

docker_mo_cmd = "docker run -it -v {}:{} iotgvmc/openvino \
              mo_tf.py \
              --input_model {} \
              --input_shape {} \
              --data_type {} \
              --output_dir {}  \
              --model_name {}".format(model_output_path, docker_mnt_path, docker_frozen_model_path, 
                                               model_inp_shape, ovData_type, docker_OvModel_Path, ovModel_name )

print("\n--".join(docker_mo_cmd.split("--")))

# Run the OpenVINO model optimizer via Docker container
!{docker_mo_cmd}

if os.path.exists(model_output_path+ovModel_path):
    print("\nOpenVINO model saved in: {}{}".format(model_output_path, ovModel_path))
    # Update permissions of the files created by docker. Change owner/group to the current user.
    update_permissions_cmd = "sudo chown $USER:$USER -R {}/{}".format(model_output_path, ovModel_path)
    !{update_permissions_cmd}

    

docker run -it -v /home/ec2-user/SageMaker/ravi/:/mnt/ iotgvmc/openvino               mo_tf.py               
--input_model /mnt/frozen_model/keras_tf_model_frozen.pb               
--input_shape [1,299,299,3]               
--data_type FP32               
--output_dir /mnt//IR_models/FP32/                
--model_name ov_inceptionv3
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/mnt/frozen_model/keras_tf_model_frozen.pb
	- Path for generated IR: 	/mnt//IR_models/FP32/
	- IR output name: 	ov_inceptionv3
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	[1,299,299,3]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	

### Step 5: Benchmark with OpenVINO benchmark app via Docker
#### [Click here for more info on OpenVINO Benchmark Tool](https://docs.openvinotoolkit.org/latest/_inference_engine_tools_benchmark_tool_README.html)

In [8]:
# Mount the directory on the docker.
# Use the OpenVINO benchmark_app to benchmark
# The benchmark_app runs the inference for 30sec with batch_size=1
docker_ovModel_xml = docker_OvModel_Path + ovModel_name + ".xml"
number_streams = 1
bench_runtime_sec = 30

docker_bench_cmd = "docker run -it -v {}:{} iotgvmc/openvino \
              python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py \
              --path_to_model {} \
              --number_streams {} \
              -nireq 1 \
              --time {}".format(model_output_path, docker_mnt_path, docker_ovModel_xml, 
                                               number_streams, bench_runtime_sec )

print("\n--".join(docker_bench_cmd.split("--")))

# Run the OpenVINO benchmark app via Docker container
!{docker_bench_cmd}


docker run -it -v /home/ec2-user/SageMaker/ravi/:/mnt/ iotgvmc/openvino               python3 /opt/intel/openvino/deployment_tools/tools/benchmark_tool/benchmark_app.py               
--path_to_model /mnt//IR_models/FP32/ov_inceptionv3.xml               
--number_streams 1               -nireq 1               
--time 30
[Step 1/11] Parsing and validating input arguments
[Step 2/11] Loading Inference Engine
[ INFO ] InferenceEngine:
         API version............. 2.1.42025
[ INFO ] Device info
         CPU
         MKLDNNPlugin............ version 2.1
         Build................... 42025

[Step 3/11] Reading the Intermediate Representation network
[ INFO ] Read network took 197.34 ms
[Step 4/11] Resizing network to match image sizes and given batch
[ INFO ] Network batch size: 1
[Step 5/11] Configuring input of the model
[Step 6/11] Setting device configuration
[Step 7/11] Loading the model to the device
[ INFO ] Load network took 663.51 ms
[Step 8/11] Setting optimal runtime para